In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


In [2]:
import cv2
from sort import Sort  # Importing the SORT tracking algorithm
import numpy as np


In [5]:

from ultralytics import YOLO
# Load YOLOv8 face detection model
model = YOLO('yolov8n-face.pt')

# Initialize SORT tracker
tracker = Sort()


# Open video file or webcam
video_path = 'test_video.mp4'  # or use 0 for webcam
cap = cv2.VideoCapture(video_path)

# Get video writer ready to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
output_path = 'output_3.mp4'
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Initialize face counting
unique_faces = set()
frame_count = 0

# Set confidence threshold
confidence_threshold = 0.1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model(frame)

    # Extract bounding boxes and confidence scores
    detections = []
    boxes = results[0].boxes  # YOLOv8 bounding boxes

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordinates
        conf = box.conf[0]  # Confidence
        cls = int(box.cls[0])  # Class (0 = face)
        
        # Apply confidence threshold
        if cls == 0 and conf >= confidence_threshold:  # Class 0 is face in the custom model
            detections.append([x1, y1, x2, y2, conf])

    # Update tracker with detections
    if len(detections) > 0:
        trackers = tracker.update(np.array(detections))
        
        # Update the set of unique face IDs
        for track in trackers:
            unique_faces.add(int(track[4]))  # Assuming the 5th element is the track ID

        # Draw bounding boxes and tracker IDs
        for track in trackers:
            x1, y1, x2, y2, track_id = map(int, track)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Save the frame with bounding boxes to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2.imshow('YOLOv8 Face Detection with Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture, video writer, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

# Print number of unique faces detected
print(f'Number of unique faces detected: {len(unique_faces)}')



c:\Users\hp\Desktop\yolo\yolov8-face\ultralytics\nn\tasks.py:362: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(weight, map_location='cpu')  # load
Ultraly

Number of unique faces detected: 131
